In [12]:
import numpy as np 
from sklearn.ensemble import VotingClassifier

In [13]:
from sklearn.datasets import load_breast_cancer

In [14]:
dataset=load_breast_cancer()
X,Y=dataset["data"],dataset["target"]
X=(X-X.min())/(X.max()-X.min())

In [15]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [16]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

### VotingClassifer from scratch

In [17]:
svm=SVC(kernel="linear",C=10,probability=True)
svm1=SVC(kernel="poly",C=100,probability=True)
svm.fit(x_train,y_train)
svm1.fit(x_train,y_train)
rfc=RandomForestClassifier(n_estimators=2,max_depth=4)
rfc.fit(x_train,y_train)

RandomForestClassifier(max_depth=4, n_estimators=2)

In [18]:
pred_svm=svm.predict_proba(x_test)
pred_svm1=svm1.predict_proba(x_test)
pred_random=rfc.predict_proba(x_test)
## soft voting 
preds=np.mean([pred_svm,pred_svm1,pred_random],axis=0)
print("Prediction")
print(np.argmax(preds,axis=1))
print("Real labels")
print(y_test)
acc=np.mean((np.argmax(preds,axis=1)==y_test))
print(acc)

Prediction
[1 0 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 0 1 0 1 0 1
 0 1 1 1 0 1 0 0 1 1 1 1 0 1 0 1 1 1 0 1 0 1 0 0 0 1 1 1 1 1 0 1 1 0 1 1 0
 0 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 0 0 0 1 1 0 1 1 1 0 0 0 1 1 1 0 1
 0 0 1]
Real labels
[1 0 1 1 0 0 0 1 1 0 1 1 1 1 0 0 1 1 1 0 0 0 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1
 0 1 1 1 0 1 0 0 1 1 1 1 0 1 0 1 1 1 0 1 0 1 0 0 0 1 1 1 1 1 0 1 1 0 1 1 0
 0 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 1 1 1 0 1
 0 0 1]
0.9473684210526315


### Hard Voting

In [19]:
from collections import Counter

In [20]:
#hard voting
votes=[]
for i in range(len(x_test)):
    pred_svm=svm.predict(x_test[i].reshape(1,-1))
    pred_svm1=svm.predict(x_test[i].reshape(1,-1))
    pred_rfc=rfc.predict(x_test[i].reshape(1,-1))
    p1=Counter(np.concatenate([pred_svm,pred_svm1,pred_rfc]))
    votes.append(max(p1))
print(votes)

[1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1]


## Using SKlearn functions 

In [22]:
vcl=VotingClassifier(estimators=[("svm",svm),("svm1",svm1),("random_forest",rfc)],voting="soft")
vcl.fit(x_train,y_train)

VotingClassifier(estimators=[('svm',
                              SVC(C=10, kernel='linear', probability=True)),
                             ('svm1',
                              SVC(C=100, kernel='poly', probability=True)),
                             ('random_forest',
                              RandomForestClassifier(max_depth=4,
                                                     n_estimators=2))],
                 voting='soft')

In [23]:
vcl.score(x_test,y_test)

0.956140350877193

## Using BaggingClassifer

In [24]:
from sklearn.ensemble import BaggingClassifier

In [26]:
bagging=BaggingClassifier(base_estimator=rfc,n_estimators=10)
bagging.fit(x_train,y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(max_depth=4,
                                                        n_estimators=2))

In [27]:
bagging.score(x_test,y_test)

0.9736842105263158

## Using Adaboost

In [28]:
from sklearn.ensemble import AdaBoostClassifier

In [29]:
ada=AdaBoostClassifier(n_estimators=10)
ada.fit(x_train,y_train)

AdaBoostClassifier(n_estimators=10)

In [30]:
ada.score(x_test,y_test)

0.9473684210526315

## Using xgboost

In [31]:
import xgboost as xgb

In [36]:
XGB=xgb.XGBClassifier()

In [37]:
XGB.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [38]:
XGB.score(x_test,y_test)

0.9824561403508771